In [1380]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter


In [1381]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
549,tt0277027,22000000,40270895,I Am Sam,Sean Penn|Michelle Pfeiffer|Dakota Fanning|Dia...,Jessie Nelson,love is all you need,Sam has the mental capacity of a 7-year-old. H...,132,Drama,New Line Cinema,12/28/2001,7.2,2001
568,tt0246989,11000000,23430766,Tomcats,Jerry O'Connell|Shannon Elizabeth|Jake Busey|H...,Gregory Poirier,The Last Man Standing Gets The Kitty.,College buddies chip in and promise that the g...,95,Comedy|Romance,Columbia Pictures|Revolution Studios,3/30/2001,4.8,2001
108,tt3077214,14000000,16002420,Suffragette,Carey Mulligan|Helena Bonham Carter|Meryl Stre...,Sarah Gavron,Mothers. Daughters. Rebels.,Based on true events about the foot soldiers o...,106,Drama|History,Ruby Films|PathÃ©|Film4,10/16/2015,7.1,2015
558,tt0179626,60000000,56359980,15 Minutes,Robert De Niro|Charlize Theron|Edward Burns|Ke...,John Herzfeld,America Likes to Watch,When Eastern European criminals Oleg and Emil ...,120,Action|Crime|Thriller,New Line Cinema|Industry Entertainment|Tribeca...,3/1/2001,5.7,2001
144,tt2788710,44000000,11305175,The Interview,James Franco|Seth Rogen|Lizzy Caplan|Randall P...,Evan Goldberg|Seth Rogen,The Film Hackers Tried To Get Banned,Dave Skylark and his producer Aaron Rapoport r...,112,Action|Comedy,Columbia TriStar,12/25/2014,6.1,2014


In [1382]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [1383]:
answers = {} # создадим словарь для ответов

# тут другие ваши предобработки колонок например:

#the time given in the dataset is in string format.
#So we need to change this in datetime format
# ...

In [1384]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


#### Check if there any non null values

In [1385]:
data.isnull().values.any()

False

#### Check if columns of type 'object' are of the right data format

In [1386]:
object_data = data.select_dtypes(include='object').columns
object_data

Index(['imdb_id', 'original_title', 'cast', 'director', 'tagline', 'overview',
       'genres', 'production_companies', 'release_date'],
      dtype='object')

In [1387]:
for ob in object_data:
    print(type(ob),'\t', ob)

<class 'str'> 	 imdb_id
<class 'str'> 	 original_title
<class 'str'> 	 cast
<class 'str'> 	 director
<class 'str'> 	 tagline
<class 'str'> 	 overview
<class 'str'> 	 genres
<class 'str'> 	 production_companies
<class 'str'> 	 release_date


#### Change columns with the wrong data format to the right one

In [1388]:
data['release_date'] = pd.to_datetime(data['release_date'])
type(data['release_date'][0])

pandas._libs.tslibs.timestamps.Timestamp

#### Format the month df to make it work easier with it

In [1389]:
def format_month_df(months):
    months.rename(columns = {'release_date':'count'}, inplace = True)
    months.index.names = ['release_month']
    month_names = ['January', 'Feburary', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'Ocotber', 'November', 'December']
    months.index = map(lambda x: month_names[x-1], months.index)
    
    return months

#### Add profit data to the dataframe

In [1390]:
data['profit'] = data['revenue'] - data['budget']
data['profit']

0       1363528810
1        228436354
2        185238201
3       1868178225
4       1316249360
           ...    
1884      82299717
1885      -8444012
1886     -40865180
1887      -3962091
1888      -9782502
Name: profit, Length: 1889, dtype: int64

#### Convert lists to pd.Series 

In [1391]:
def list_to_Series(x): 
    idx, values = zip(*x)
    return pd.Series(values, idx)

#### Split data into lists  

In [1392]:
def to_one_str(column):
    return column.str.cat(sep = '|').split('|')

#### Find most_common word in a string

In [1393]:
def most_common(column):
    return Counter(to_one_str(column)).most_common()

#### Get production studios sorted according to the passed criteria

In [1394]:
def get_studios(col):
    studios = df.groupby(['production_companies']).median(col)
    studios = studios.sort_values(by = col, ascending = False)

    return studios

#### Calculate length of strings contained in the passed column

In [1395]:
def calc_len(column):
    return list(map(lambda x: len(x), column))

#### Collect an answer to questions

In [1396]:
answers = {}

# id_name is True if the input contains both name and id
def answer(answer, question, id_name = False):
    if id_name:
        answers[question] = answer['original_title'].item() + ' (' + answer['imdb_id'].item() + ')'
    else: 
        answers[question] = answer
answers

{}

# 1. У какого фильма из списка самый большой бюджет?

In [1397]:
max_budget = data[data['budget'] == data['budget'].max()]
answer(max_budget, 1, True)

answers[1]

'Pirates of the Caribbean: On Stranger Tides (tt1298650)'

# 2. Какой из фильмов самый длительный (в минутах)?

In [1398]:
longest = data[data['runtime'] == data['runtime'].max()]
answer(longest, 2, True)

answers[2]

'Gods and Generals (tt0279111)'

# 3. Какой из фильмов самый короткий (в минутах)?





In [1399]:
shortest = data[data['runtime'] == data['runtime'].min()]
answer(shortest, 3, True)

answers[3]

'Winnie the Pooh (tt1449283)'

# 4. Какова средняя длительность фильмов?


In [1400]:
avg_runtime = data['runtime'].mean()
answer(avg_runtime, 4)

answers[4]

109.6585494970884

# 5. Каково медианное значение длительности фильмов? 

In [1401]:
meadian_runtime = data['runtime'].median()
answer(meadian_runtime, 5)

answers[5]

107.0

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [1402]:
most_profit = data[data['profit']==data['profit'].max()]
answer(most_profit, 6, True)

answers[6]

'Avatar (tt0499549)'

# 7. Какой фильм самый убыточный? 

In [1403]:
least_profit = data.query('profit == profit.min()')
answer(least_profit, 7, True)

answers[7]

'The Lone Ranger (tt1210819)'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [1404]:
pos_profit = data[data['profit']>0].count()['imdb_id']
answer(pos_profit, 8)

answers[8]

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [1405]:
movies_2008 = data.query('release_year == 2008')
highest_gross =  movies_2008.query('revenue == revenue.max()')
answer(highest_gross, 9, True)

answers[9]

'The Dark Knight (tt0468569)'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [1406]:
period = data[data['release_year'].between(2012,2014)]

most_loss = period.query('profit == profit.min()')
answer(most_loss, 10, True)

answers[10]

'The Lone Ranger (tt1210819)'

# 11. Какого жанра фильмов больше всего?

In [1407]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале
genres = data.groupby(['genres']).size().sort_values(ascending=False)
most_popular = genres[0]
print(genres)

answer(genres.index[0], 11)

answers[11]

genres
Comedy                            141
Drama                             102
Comedy|Romance                     60
Drama|Romance                      45
Comedy|Drama|Romance               35
                                 ... 
Drama|Mystery|Thriller|Fantasy      1
Drama|Mystery|Thriller|Horror       1
Drama|Romance|Comedy                1
Drama|Romance|Comedy|Music          1
Crime|Comedy|Romance                1
Length: 652, dtype: int64


'Comedy'

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [1408]:
profitable = data[data.profit > 0]
quantity = most_common(profitable['genres'])

display(quantity)

answer(quantity[0][0], 12)

[('Drama', 560),
 ('Comedy', 551),
 ('Thriller', 446),
 ('Action', 444),
 ('Adventure', 337),
 ('Romance', 242),
 ('Crime', 231),
 ('Family', 226),
 ('Science Fiction', 195),
 ('Fantasy', 188),
 ('Horror', 150),
 ('Animation', 120),
 ('Mystery', 119),
 ('Music', 47),
 ('History', 46),
 ('War', 41),
 ('Western', 12),
 ('Documentary', 7)]

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [1409]:
directors = data.groupby(['director'])
movie_shark = directors['revenue'].sum().sort_values(ascending = False)
answer(movie_shark.index[0], 13)

movie_shark.head(5)

director
Peter Jackson        6490593685
Christopher Nolan    4167548502
David Yates          4154295625
Michael Bay          3886938960
J.J. Abrams          3579169916
Name: revenue, dtype: int64

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [1410]:
action = data[data['genres'].str.contains('Action')][['director','genres']]
directs_most = list_to_Series(most_common(action['director']))

display(directs_most.head())

answer(directs_most.index[0],14)

Robert Rodriguez      9
Michael Bay           7
Paul W.S. Anderson    7
Antoine Fuqua         6
Ridley Scott          6
dtype: int64

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [1411]:
movies_12 = data[data['release_year']==2012][['original_title', 'cast', 'revenue']]

top_movies = movies_12.query('revenue > revenue.mean()').sort_values(['revenue'],ascending=False)
pop_actors = list_to_Series(most_common(top_movies['cast']))

display(pop_actors.head())

answer(pop_actors.index[0], 15)

Chris Hemsworth        2
Ralph Fiennes          2
Anne Hathaway          2
Denis Leary            2
Seann William Scott    2
dtype: int64

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [1412]:
high_budget = data.query('budget > budget.mean()')[['original_title','cast','budget']]
most_pop_actor = list_to_Series(most_common(high_budget['cast']))

answer(most_pop_actor.index[0], 16)
most_pop_actor.head(3)


Matt Damon        18
Adam Sandler      17
Angelina Jolie    16
dtype: int64

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [1413]:
cages_movies = data[data['cast'].str.find('Nicolas Cage')>=0][['original_title', 'cast', 'genres']]

cages_pop_genre = list_to_Series(most_common(cages_movies['genres']))

answer(cages_pop_genre.index[0], 17)
cages_pop_genre.head()

Action      17
Thriller    15
Drama       12
Crime       10
Fantasy      8
dtype: int64

# 18. Самый убыточный фильм от Paramount Pictures

In [1414]:
paramount = data[data['production_companies'].str.find('Paramount Pictures')>=0]
paramount = paramount[['imdb_id','original_title','profit']]

unsuccessful = paramount.query('profit == profit.min()')

answer(unsuccessful, 18, True)
answers[18]

'K-19: The Widowmaker (tt0267626)'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [1415]:
revenue = data.groupby('release_year')['revenue'].sum().sort_values(ascending=False)
display(revenue.head())

answer(revenue.index[0], 19)
answers[19]

release_year
2015    25449202382
2014    23405862953
2013    23213799791
2012    23079001687
2011    22676791872
Name: revenue, dtype: int64

2015

# 20. Какой самый прибыльный год для студии Warner Bros?

In [1416]:
WB_movies = data[data['production_companies'].str.find('Warner Bros')>0][['release_year','profit']]

most_profit_year = WB_movies.groupby('release_year').sum().sort_values(['profit'],ascending=False)

display(most_profit_year.head(3))

answer(most_profit_year.index[0], 20)
answers[20]

,profit
release_year,
2014,2297979392
2008,2134595031
2007,1928180689


2014

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [1417]:
months = data['release_date'].apply(lambda x: x.month)
months = format_month_df(months.groupby(months).count().to_frame())

highest_num = months.sort_values(by = ['count'], ascending = False).index[0]

answer(highest_num, 21)
answers[21]

'September'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [1418]:
values = months[5:8]['count'].values
summer_mov = sum(values)

answer(summer_mov, 22)
answers[22]

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [1419]:
df = data[['release_date','director']].copy()
df['release_date'] = df['release_date'].apply(lambda x: x.month)

winter = [12,1,2]
df = df[df['release_date'].isin(winter)]

### Вариант 1: использование counter

In [1420]:
winter_directors = most_common(df['director'])

answer(winter_directors[0][0], 23)
answers[23]

'Peter Jackson'

### Вариант 2: использование explode

In [1421]:
df['director'] = split(df['director'])
df = df.explode('director')

winter_directors = df.groupby(['director']).count().sort_values(by= 'release_date', ascending = False)

answer(winter_directors.iloc[0].name, 23)
answers[23]

'Peter Jackson'

# Questions 24 and 25

In [1429]:
df = data['production_companies'].copy().to_frame()
df.insert(1, "title_len", calc_len(data['original_title']), True)
df.insert(2, "overview_len", calc_len(data['overview']), True)

df['production_companies'] = split(df['production_companies'])
df = df.explode('production_companies')

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [1423]:
studios = get_studios('title_len')

display(studios)

answer(studios.iloc[0].name, 24) 

,title_len,overview_len
production_companies,,
Four By Two Productions,83.0,298.0
"Jim Henson Company, The",59.0,304.0
Dos Corazones,47.0,150.0
Museum Canada Productions,46.0,302.0
Polsky Films,46.0,387.0
...,...,...
Everest Entertainment,3.0,124.0
Berlanti Productions,3.0,413.0
Ixtlan Productions,2.0,340.0


# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [1424]:
studios = get_studios('overview_len')

display(studios)

answer(studios.iloc[0].name, 25) 

,title_len,overview_len
production_companies,,
Midnight Picture Show,23.0,1000.0
Room 9 Entertainment,21.0,964.0
Brookwell-McNamara Entertainment,11.0,936.0
Lions Gate Family Entertainment,15.0,909.0
Crest Animation Productions,15.0,909.0
...,...,...
Projection Pictures,16.0,74.0
London Boulevard,16.0,74.0
Phantom Four,10.0,72.0


# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 

In [1425]:
best_movies = data[['imdb_id', 'original_title', 'vote_average']].sort_values('vote_average', ascending = False)
quantity = len(best_movies)
one_percent = round(quantity * 0.01)

display(best_movies[:one_percent])

answer(list(best_movies[:one_percent]['original_title'].values), 26)

,imdb_id,original_title,vote_average
599,tt0468569,The Dark Knight,8.1
118,tt0816692,Interstellar,8.0
125,tt2084970,The Imitation Game,8.0
9,tt2096673,Inside Out,8.0
34,tt3170832,Room,8.0
1183,tt0993846,The Wolf of Wall Street,7.9
128,tt2267998,Gone Girl,7.9
1191,tt2024544,12 Years a Slave,7.9
119,tt2015381,Guardians of the Galaxy,7.9
1081,tt0167260,The Lord of the Rings: The Return of the King,7.9


# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [1426]:
from itertools import combinations

actors = data[['original_title', 'cast']].copy()

actors['cast'] = split(actors['cast'])
actors['combinations'] = actors['cast'].apply(lambda cast: list(combinations(cast,2)))

actors = actors.explode('combinations')

often_together = actors.groupby('combinations').count().sort_values('cast',ascending= False)
the_pairs = list(often_together.query('cast == cast.max()').index)

answer(the_pairs, 27)
answers[27]

[('Daniel Radcliffe', 'Rupert Grint'), ('Daniel Radcliffe', 'Emma Watson')]

# Submission

In [1427]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{1: 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 2: 'Gods and Generals (tt0279111)',
 3: 'Winnie the Pooh (tt1449283)',
 4: 109.6585494970884,
 5: 107.0,
 6: 'Avatar (tt0499549)',
 7: 'The Lone Ranger (tt1210819)',
 8: 1478,
 9: 'The Dark Knight (tt0468569)',
 10: 'The Lone Ranger (tt1210819)',
 11: 'Comedy',
 12: 'Drama',
 13: 'Peter Jackson',
 14: 'Robert Rodriguez',
 15: 'Chris Hemsworth',
 16: 'Matt Damon',
 17: 'Action',
 18: 'K-19: The Widowmaker (tt0267626)',
 19: 2015,
 20: 2014,
 21: 'September',
 22: 450,
 23: 'Peter Jackson',
 24: 'Four By Two Productions',
 25: 'Midnight Picture Show',
 26: ['The Dark Knight',
  'Interstellar',
  'The Imitation Game',
  'Inside Out',
  'Room',
  'The Wolf of Wall Street',
  'Gone Girl',
  '12 Years a Slave',
  'Guardians of the Galaxy',
  'The Lord of the Rings: The Return of the King',
  'Memento',
  'Inception',
  'The Pianist',
  'The Grand Budapest Hotel',
  'Her',
  'Spotlight',
  'Big Hero 6',
  'The Fault in Our Stars'

In [1428]:
# и убедиться что ни чего не пропустил)
len(answers)

27